In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

Using TensorFlow backend.


In [7]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 40, 40, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta',metrics=['accuracy'])


In [8]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1

                # Make it more robust by adding noise.
                # The idea is that if during inference,
                # the value of the pixel is not exactly one,
                # we need to train the network to be robust and still
                # consider it as a pixel belonging to a square.
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

In [9]:
noisy_movies, shifted_movies = generate_movies(n_samples=1200)

In [10]:
noisy_movies.shape

(1200, 15, 40, 40, 1)

In [ ]:
seq.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10,
        epochs=300, validation_split=0.05)

Train on 950 samples, validate on 50 samples
Epoch 1/300
950/950 [==============================] - 1330s 1s/step - loss: 0.2968 - acc: 0.9018 - val_loss: 0.0729 - val_acc: 0.9897
Epoch 2/300
950/950 [==============================] - 1370s 1s/step - loss: 0.0272 - acc: 0.9952 - val_loss: 0.0131 - val_acc: 0.9984
Epoch 3/300
950/950 [==============================] - 1313s 1s/step - loss: 0.0045 - acc: 0.9993 - val_loss: 0.0036 - val_acc: 0.9994
Epoch 4/300
950/950 [==============================] - 1316s 1s/step - loss: 0.0015 - acc: 0.9997 - val_loss: 0.0014 - val_acc: 0.9996
Epoch 5/300
950/950 [==============================] - 1312s 1s/step - loss: 9.0439e-04 - acc: 0.9998 - val_loss: 8.3010e-04 - val_acc: 0.9998
Epoch 6/300
950/950 [==============================] - 1476s 2s/step - loss: 6.2705e-04 - acc: 0.9998 - val_loss: 7.3807e-04 - val_acc: 0.9998
Epoch 7/300
950/950 [==============================] - 2006s 2s/step - loss: 5.2261e-04 - acc: 0.9999 - val_loss: 5.8172e-04 - va